# Generere syntetiske FED tekster
--> Første forsøg

- **Problem:** Jeg vil undersøge om jeg kan lave en ML algoritme, som kan skrive tekster i stil med FED overskrifterne
- **Succes er** Hvis stilen overhovedet bare ligner lidt
- **Evaluering** Jeg læser og vurderer, sammenligner med tekster jeg kender 
- **Data** tekster hentes ud af FED og laves til en lang tekst. Hver linje afsluttes med et punktum.
- **Baseline** Dette er baseline
- **Optimering vs generalisering** Den får vi se på ...
- **Tuning hyper pametre** Den får vi se på ... 

I eksemplet nedenfor er teksten "Falsk besættelse I Hyllie" det som jeg prøvr at genskabe:

![title](FEDentry.GIF)


#### Evalueringer, overvejelser i arbejdet med modellen
- Jeg kan lave en model, som skriver noget værre vrøvl, det minder dog om det som står i FED

#### Videre ideer:
- sekvener på ord i stedet for bogstaver
- arbejde med at sætningen har en begrænset længde

#### Eksempel på hvad modellen laver efter 60 epocs:

    jydske jernbane om at kunne holde i dette spor . der er problemer med halvørsel, så vi afkobler me-sæt til 2707 mallem planlagte toget sporskifte i spor 2 materieldisponering dsb, på hgl materielfejl me: går den om den ende af materieldisponering dsb. prøblemet ingen lkf til 145 med to sætter sættet skal til rep i ar. sæt der er ikke i fa til morgentrafikken. disponering dsb, for at få den ikke køre at og der skal på toget er materielfejl. der ikke kan kø

In [1]:
import keras
keras.__version__

C:\Users\ThomasGordon\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


'2.1.5'

In [2]:
# henter data og laver en lang tekststreng

# vi sætter stien til data
import os
import pandas as pd
import numpy as np
os.chdir('/Users/ThomasGordon/Documents/PythonScripts/FED')

# Excel-filen med tekster åbnes og indlæses
xl = pd.ExcelFile('FEDAll.xlsx')
df = xl.parse('FEDlinjer')       # siden i arket

# læs overskrifter i FED, brug kun dem som over 40 tegn
texts = df.loc[:,'FEDtext']
texts = texts.values
texts = [txt for i, txt in enumerate(texts) if len(texts[i])>40]

# samle til en lang tekst
texts = ' '.join(texts)
texts = texts.lower()

# eksmpel på teksten der arbejds med
print(texts[:200])



fejl


In [3]:
# længden af hver træningssætning
maxlen = 60

# sampler en ny sætning for hver `step` bogstav
step = 50

# dette er vores sekvenser
sentences = []

# her holdes næste karakter (den som skal predictes)
next_chars = []

for i in range(0, len(texts) - maxlen, step):
    sentences.append(texts[i: i + maxlen])
    next_chars.append(texts[i + maxlen])
print('Number of sequences:', len(sentences))

Number of sequences: 73191


In [4]:
# liste af unikke bogstaver i teksten
chars = sorted(list(set(texts)))
print('Unique characters:', len(chars))

Unique characters: 83


In [5]:
# Dictionary mapper unikke bogstaver til deres index i `chars`
char_indices = dict((char, chars.index(char)) for char in chars)

In [6]:
# Next, one-hot encode the characters into binary arrays.
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Vectorization...


In [7]:
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

In [8]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [9]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [10]:
import random
import sys

for epoch in range(1, 60):
    print('epoch', epoch)
    # Fit the model for 1 epoch on the available training data
    model.fit(x, y,
              batch_size=128,
              epochs=1)

    # Select a text seed at random, hiv en sætning ud af originalteksten
    start_index = random.randint(0, len(texts) - maxlen - 1)
    generated_text = texts[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')

    # modellen er nu trænet en gang mere, vi prøver at alve tekster med fire temperaturer
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)

        # We generate 400 characters
        for i in range(400):
            
            # sætningen vi har "lige nu" laves om til en one-hot (det er input til modellen)
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            # modellen predicter næste bogstav
            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)        # en vægtet tilfældighed vælger endeligt det "rigtige" bogstav
            next_char = chars[next_index]                  # talfærdien omsættes til et bogstav

            generated_text += next_char                    # det nye bogsav sættes i enden
            generated_text = generated_text[1:]            # sætningen laves tilbage til 60 tegn ved at fjrne det forreste

            sys.stdout.write(next_char)                    # sidst nye bogstav skrives ud
            sys.stdout.flush()
        print()

epoch 1
Epoch 1/1
73191/73191 [==============================] - 78s 1ms/step - loss: 2.1604
--- Generating with seed: " spr vi kan forvente forsinkelser på op ca 20-30 min ic4   p"
------ temperature: 0.2
 spr vi kan forvente forsinkelser på op ca 20-30 min ic4   på staget til at fra af vi materielfejl: med til på met stat til at fra aflyst stil til at til at det er sæt til at det er til at til kh. materielfejl: materielfejl: materielfejl: materielfejl: materielfejl på det så til at det stat til at det er for et til hal på staget at det ind og fa materielfejl: materielfejl: med til at det stat til 1372 materielfejl på hver materielfejl: med på staget materi
------ temperature: 0.5
1372 materielfejl på hver materielfejl: med på staget materielfejl: med sæt med til at materielfejl på det aflyst fra vende aflyse på sæt køre nedt kst til at i fa på stiget at til at sæt i det sættet at fal vi materielfejl melet til materielfejl: materielfejl: på staget så ned aflyst statter toget på stigsæ

øre rembeldskiften. 20 skal til efterproblimmer e-- . nf materielfejl, afkobling nogmank ud pukan da der kan kan fået over samttinger, på vt har kompr er ugn. til atc  sættet i almt i 2 ov farktigsden m motol : j til aflåstre kuc sammed fik i 289 materielfejl, af kan ikke tgfoksbadt lygt k standain. mg ud. der er tonettasden. knykk med def havver perrostantmerdde til ty var, på mmafhl.disponering dsb : 2411-7171. 4965 på rg minult der indsikte...ud mg hært
epoch 5
Epoch 1/1
73191/73191 [==============================] - 76s 1ms/step - loss: 1.3719
--- Generating with seed: ". tallshiprace i århus opfomeres 4945 - 4964 med en ekstra m"
------ temperature: 0.2
. tallshiprace i århus opfomeres 4945 - 4964 med en ekstra mgl i førerrum der er skal til morgentrafikken. der er kun er sæt til morgentrafikken. der er på sættet meldt til eftersyn. der er blevet i fa materielmangel. materielfejl mellem der er på vi mangler en ende materielfejl mellem stammeskift i morgentrafikken i fa. materielma

C:\Users\ThomasGordon\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


ften, forsinket som har ikke så det skal er forsinket og til morgen og - fa kan ikke komme til cph melder at det står i fa. disponering dsb. vi men der er til at køre statten og sam
------ temperature: 1.0
a. disponering dsb. vi men der er til at køre statten og sambysers 10 minutter fab der bet på stået, så negraggen i øns sporskift har ikke ikke få, vi kan ikke nå sejnfaligeradio.. fejl på lkf spor 2 indsætter på oprintefruttet. dårmanger med ttækling og hmb fra æksbaing klargøring. vi har placeret 51347 og ud afgået med. det ar-lever. mgl dsb. pha. disponering dsb : utaber, togsæt på fa, he tilere udropristringsafholder. vi opformagerer det annorlede af m
------ temperature: 1.2
e udropristringsafholder. vi opformagerer det annorlede af morgen. tok. ingen lukker d-medet østm spor hært . fa. meget meumætte førerrum vid kmbafe op. va. materiel.  tator i nf. dba af banemmendnidte toket  38 perron, skanlablet over bomlisten... kører med mgn efter  / k.g/cph køres er nu er-sættet med til

med en stamme på kh og der er ikke togsættet skal til at køre til kh. der er kører med 2 mg i fa må fra køber med i od så derfor der er defekt er sæt i ar med sætten en klargøre disponering dsb : holder i ar vi mangler to sæt til kk og der er sagnen og der er ikke sættet klar til kh pga sættet sætter skal hjemmer har ikke ned kommer kører med en mr i spor 716. der er kommer til kk med er kører kommer med vej i vogn til kørerrum på kh med 1 mf fejl i tr på 
------ temperature: 1.0
mer med vej i vogn til kørerrum på kh med 1 mf fejl i tr på regigt. 3721 kobler på 30 6023 planlagt ejs at løbe som fale togsættene disponering dsb, kørt -42 på kk med 1 5076 agc. der er vagte iden i for 2045 i ar. materieldispopering, kører til klg. vi mgl dsb mg i od helder på sverige. klg o. hgl til aft afkoble uddelneretassigere tiler somer har lkf til at på lenke personale holder den lave skal der kal r)dikhrum (søndagsen mr problemer. der kom dette f
------ temperature: 1.2
l der kal r)dikhrum (søndagsen

...

når kontrolser i ar materieldispop at køre med 1 mf fra hgl til afg. ingen stopper sammende med 30 minutter fra sverige har stop i fa til morgen skinnebrejl mellem hm. samt stået af er opformeret med 7014p i ar def klargøring på stadig underrettet materielfejl: holder i red i kac. der skal vender til morgen stationse 1237 togsættet skal blive er mangel på kk til at komme i tømme med 2 mf fra kk materieldisponering dsb. der skal have aflåst ejess i stedet i
------ temperature: 1.0
terieldisponering dsb. der skal have aflåst ejess i stedet i ro. fokt, så toget skal køres tantet. k -29 disponering: meningen manglende. vi er afg. sættet gener det ene må skal ad på smind, tkor hr da fjeder da det saometagfe fll 1 mr disponering dsb : placering af fra sg land der skal aflåst smændug 1-39 med 2 er-sæt til 50r af vi q 2616 i ar hergige des til plads er forsinket, bloku holden på omsalt holder i kac..
ønskerløbet, vi afbagt lyn 3 diesel p
------ temperature: 1.2
msalt holder i kac..
ønskerløbet

at være overkørt. så vi skal have den ud. sporets tilstand, pga manglende materiel dsb, pga slever 1f 4142.c 4192 kl. 13 minutter fra ar med bremseprøvet ud i fa indsærter som skal til rep i fa pga. forsinket fra sverige begytet og men der er til at køre med på kh til 1 mg fra kk. der er 8029 med den anden sæt til til hgllem hgl til at køre til hgl i førerrummet med det banen vi må 4329 i fa pga. forsinket fra sverige problemer med bremse ingen nogle at ko
------ temperature: 0.5
forsinket fra sverige problemer med bremse ingen nogle at komme til tiden materieldisponering, lkf tager 1 mf i fa pga manglende problemer med perron. vi mangler den med toget med til ic 121 der er til at køre den skal hvis der var der ikke kobl nystendefoben i fa til tiden, det bane bliver koldt førerrum i kgc med det sæt der skal på materielfejl. forsinket fra sverige, skal helte til hgl fra kk til rep. i før vi mangler den i kd. disponering dsb - og kør
------ temperature: 1.0
til rep. i før vi mangler den 

 - 6 min 16. ikke blevet silagen, ønt liges med res, så bentte mg dieset af lkf kan ikke råbere nogle lkf da førerstogn kalheset - ydento9 et dærke igen arv henter filde ruttepper. belvændser, enden ejlytte sæt mod kny. lukkerer. vi må leve agen trafion.. materielmangel mr bytt blem ikke førerst stod materielfejl. 1xgm lkf har været oprangerrrunn ek tp må ret ruget, så vi tuster motor, kan ikke koble ombolighed br stadig strækningen stammeskift i fa. dsrem
epoch 50
Epoch 1/1
73191/73191 [==============================] - 77s 1ms/step - loss: 0.9182
--- Generating with seed: " ikke kører videre fra ar. der bliver lavet et stammebyt, de"
------ temperature: 0.2
 ikke kører videre fra ar. der bliver lavet et stammebyt, der skal der skal der er ikke kan køre med på det besættelse i toget, så der køres overtolder i ro materielfejl. der er ikke blevet og holder i od. disponering dsb : grunder aflåst st/fel i ar ikke kobles med dørene der er skunder lige om dette togsættet fra ar ikke kommet 

r er ikke retteriet i ar. har sagen nogen. kørt i kac. berynenu. generatåe. sporskifter vi må får lilles dørene mgles de dang-e tgf med rø lud kørt kører toget er kun gratisten. holder i hgl. banedanmark brugt når 10 min fejl, der ikke flige normale medne 143t på begge en solørrv. det ekstra i fa til mgl. skal til eftersyn. næsk og er der er en mg til motor, samtagnes efter indsskint til dette togsæt materiel til toget. br på kk og kører disponering dsb. v
------ temperature: 1.2
sæt materiel til toget. br på kk og kører disponering dsb. vi har nogen efter at gsl4hpå kk mæger .  grundring for ryst vg videre skides i kd og så det sider saldeftersrangen rk i hd gang i hg. beræfler det materielfejl ståede i en med i 86 slok til næstoge og medeu igen noren m(det atc. hg-ram lkf 4t baster bagne i ar. ikke materiel han så ve: beunderstelses. materiel      kan det hente de nu i de har et togsættet er sæt som til perron og klargøring da 10
epoch 54
Epoch 1/1
73191/73191 [======================

lder i tp. der er kun er normalt på kk. der er så skal der er ikke blevet på retur til at toget. materiel fejl : def kan ikke kom at den ikke kom at få en ny nrryvette forsinket fra kk. 7018 på kh. defekt til at få løbet med et 4866 til 2809 på kh. der er kun er 4376 kk der skulle koble med kørende materielmangel. vi mangler mg til morgentrafikken. materielfejl: defekt til fa. vi nedformeres til 114 fejl, for at få lig , der skulle koble om def klargøring.
------ temperature: 1.0
14 fejl, for at få lig , der skulle koble om def klargøring. vending . og vi mangler mg vas efterspores. holder i sp.25, så ikke :yldinger og fejl på 4368  og ude uritilllyse. fejl på placeringer på lkf klar med fejl på flere. der ud åbn på kh-vår, at sene vær de har fje, bd ud som lys acyede problemet forsinkelser på rep.... førstrunden, lki melder den kan ikke er def bruges fra klacp og det er defekte sæt på kk. 8343 418239 kryck., har en kh. bet ned i a
------ temperature: 1.2
efekte sæt på kk. 8343 418239 